In [2]:
# Importing the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib as plb
import seaborn as sns
from scipy import stats
import statistics
import dabest as db

#### Reading in the data needed to generate  the table

In [13]:
GNmm_df = pd.read_csv('C:/Users/Emily/Documents/S1F2/GN960_xs.csv', index_col=0)
CXmm_df = pd.read_csv('C:/Users/Emily/Documents/S1F2/CX10_xs.csv',  index_col=0)
PRmm_df = pd.read_csv('C:/Users/Emily/Documents/S1F2/PR678_xs.csv',  index_col=0)
n2 = pd.read_csv('C:/Users/Emily/Documents/S1/S1_xs3.csv',  index_col=0)
all_strains_nlref = pd.read_csv('C:/Users/Emily/Documents/S1F2/all_strains_nullrefs_mdiff.csv', index_col = 0)

#### Mean difference caluclations for between strains analysis
This code block is here just to show how the between species bootstraped mean difference confidence intervals were generated

In [14]:
#Need to pass a list to the dabest package.
cols = list(GNmm_df.columns)
#The "control" condition needs to be the first item in the list
cols.insert(0, cols.pop(cols.index('DMSO')))

temp = pd.DataFrame()
N2_all = pd.DataFrame()

strains = ['PR678', 'CX10', 'GN960']
refs = ['DMSO', 'H2O']

for s in strains:
    if s == 'CX10':
        xs = CXmm_df
    elif s == 'PR678':
        xs = PRmm_df
    elif s == 'GN960':
        xs = GNmm_df
    for c in cols:
        temp['N2'] = n2[c]
        temp[s] = xs[c]
        db_obj = db.load(temp, idx=(['N2', s]))
        hold = db_obj.mean_diff.results
        hold['Compound'] = c
        N2_all = N2_all.append(hold)
    

#### Generating a function to return analyses results
We want to return a symbol to represent the results from 2 seperate analyses:
1. Between strains analysis: Is there a difference between the N2 and mutant response for each test condition?
2. Between compounds analysis: For each mutant strain, is the response for each test condition different from the response to the null reference condition?

In [23]:
def cat_dat(s, df, df2):
    # Checking to see if the mutant response is similar to the null reference condition response. If so
    # then we can assume that there has been a loss of behavioral phenotype in the mutant strain
    x = df.loc[(df['Strain'] == s)  & (df['bca_low']<0) 
                             & (df['bca_high']>0)]
    
    # There may be duplicates in the list because we are looking at the data for 2 null reference conditions DMSO:DMSO and
    # DMSO:H2O. 
    c = x['test'].unique().tolist()

    # Checking to see if the mutant response is similar to the N2 response
    y = df2.loc[(df2['test'] == s)  & (df2['bca_low']<0) & (df2['bca_high']>0)]
    
    z= y['Compound'].unique().tolist()

    return c, z


### Generating a dataframe that can made in to a table to report the results above
srtd = pd.read_csv('C:/Users/Emily/Documents/S1F2/sig_mut.csv')
srtd_cmps = srtd['Compound']

nlref = pd.DataFrame(columns=['Compound', 'GN960', 'PR678', 'CX10'])
nlref['Compound'] = srtd_cmps


In [25]:
for s in strains:
    match_null, match_n2  = cat_dat(s, all_strains_nlref, N2_all)

    for index, row in nlref.iterrows():
        #if the compound test condition is the same as DMSO and the n2 response is different from the mutant response,
        #then return * and +
        if (row['Compound'] in match_null) & (row['Compound'] not in match_n2):
            row[s] = '*  +'
        #if the compound test condition response is different from the null condition response, but there is no discernable difference
        # in the mutant response when compared to N2, only return *
        elif (row['Compound'] in match_null) & (row['Compound'] in match_n2):
            row[s] = '*'
        #if there is only a differnce in between the N2 and mutant response to the same condition, but the response is still 
        #different between the null reference and test conditions, return +
        elif (row['Compound'] not in match_null) & (row['Compound'] not in match_n2):
            row[s] = '+'

['Cinnamyl Alcohol', 'Ethyl p-methoxycinnamate', 'Lapachol', 'Piperonyl Alcohol', 'Sabinene', 'Sinomenine hydrochloride', 'Solasodine']
['DMSO', '2,3-Dihydrobenzofuran', '2-Methyl-1-butanol', '2-nonanone', 'Anisole', 'Daucosterol', 'Furfural', 'H2O', 'Limonin', 'Oleanolic Acid', 'Phenylacetylene', 'Piperitenone', 'Sabinene', 'Ursolic acid', 'p-Tolualdehyde']
['DMSO', 'H2O', 'Ursolic acid']


In [26]:
nlref

,Compound,GN960,PR678,CX10
0,Isoamyl alcohol,* +,* +,+
1,2-Methyl-1-butanol,* +,+,NaN
2,Thiophene,* +,* +,* +
3,"2,3-Dihydrobenzofuran",* +,+,NaN
4,Diacetyl,* +,+,+
5,Acetophenone,+,+,* +
6,Phenylacetylene,* +,* +,NaN
7,Paeoniflorin,* +,* +,+
8,α-Phellandrene,* +,* +,* +
9,Coumaran,* +,* +,* +
